In [14]:
import pandas as pd
import numpy as np

In [15]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [16]:
from IPython.display import HTML

In [17]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(50, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(d1)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model

model = createModel()

In [19]:
model_string = f"models/fifth_model/weights_epochs20"

model.load_weights(model_string)

In [20]:

x_val = np.load("./cleaned_data/x_val.npy")
y_val = np.load("./cleaned_data/y_val.npy")
x_test = np.load("./cleaned_data/x_test.npy")
y_test = np.load("./cleaned_data/y_test.npy")
train_mu = np.load("./cleaned_data/train_mu.npy")
train_std = np.load("./cleaned_data/train_std.npy")


In [21]:
preds = model.predict(x_val[:,:,4:])

1317/1317 [==============================] - 2s 2ms/step


In [22]:
print(x_val.shape)
print(preds.shape)

(42131, 23, 11)
(42131, 3)


In [23]:
y_val[0:10]

array([[ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.]])

In [24]:
preds[0:10]

array([[ 0.54610014,  0.4538999 , -0.73073393],
       [ 0.5256712 ,  0.47432882, -1.0027636 ],
       [ 0.5039362 ,  0.49606383, -1.0908389 ],
       [ 0.5251753 ,  0.47482467, -0.4746756 ],
       [ 0.5518019 ,  0.44819802, -0.242254  ],
       [ 0.56256485,  0.4374351 , -0.02373301],
       [ 0.5186323 ,  0.48136774,  0.20763762],
       [ 0.54657406,  0.45342594,  0.17305745],
       [ 0.45594263,  0.5440574 ,  0.27965552],
       [ 0.5026888 ,  0.49731112,  0.011212  ]], dtype=float32)

In [25]:

unadj_x_val = x_val
unadj_x_val[:,:,5:] = (x_val[:,:,5:]*train_std[5:])+train_mu[5:]
unadj_x_val[:,:,6] += (53.3/2)
        

In [26]:
unadj_x_val[0][0]

array([2.0211017e+09, 5.2100000e+02, 6.0000000e+00, 3.8544000e+04,
       1.0000000e+00, 8.6660000e+01, 3.5350000e+01, 0.0000000e+00,
       7.0000000e-02, 1.7022000e+02, 1.5666000e+02])

In [27]:
small = preds[0:4]
print(f"small shape = {small.shape}")
print(small)

small shape = (4, 3)
[[ 0.54610014  0.4538999  -0.73073393]
 [ 0.5256712   0.47432882 -1.0027636 ]
 [ 0.5039362   0.49606383 -1.0908389 ]
 [ 0.5251753   0.47482467 -0.4746756 ]]


In [28]:
np.repeat(small, 3, axis=0)

array([[ 0.54610014,  0.4538999 , -0.73073393],
       [ 0.54610014,  0.4538999 , -0.73073393],
       [ 0.54610014,  0.4538999 , -0.73073393],
       [ 0.5256712 ,  0.47432882, -1.0027636 ],
       [ 0.5256712 ,  0.47432882, -1.0027636 ],
       [ 0.5256712 ,  0.47432882, -1.0027636 ],
       [ 0.5039362 ,  0.49606383, -1.0908389 ],
       [ 0.5039362 ,  0.49606383, -1.0908389 ],
       [ 0.5039362 ,  0.49606383, -1.0908389 ],
       [ 0.5251753 ,  0.47482467, -0.4746756 ],
       [ 0.5251753 ,  0.47482467, -0.4746756 ],
       [ 0.5251753 ,  0.47482467, -0.4746756 ]], dtype=float32)

In [36]:
np.tile(small, (1,4)).reshape(-1,4,3)

array([[[ 0.54610014,  0.4538999 , -0.73073393],
        [ 0.54610014,  0.4538999 , -0.73073393],
        [ 0.54610014,  0.4538999 , -0.73073393],
        [ 0.54610014,  0.4538999 , -0.73073393]],

       [[ 0.5256712 ,  0.47432882, -1.0027636 ],
        [ 0.5256712 ,  0.47432882, -1.0027636 ],
        [ 0.5256712 ,  0.47432882, -1.0027636 ],
        [ 0.5256712 ,  0.47432882, -1.0027636 ]],

       [[ 0.5039362 ,  0.49606383, -1.0908389 ],
        [ 0.5039362 ,  0.49606383, -1.0908389 ],
        [ 0.5039362 ,  0.49606383, -1.0908389 ],
        [ 0.5039362 ,  0.49606383, -1.0908389 ]],

       [[ 0.5251753 ,  0.47482467, -0.4746756 ],
        [ 0.5251753 ,  0.47482467, -0.4746756 ],
        [ 0.5251753 ,  0.47482467, -0.4746756 ],
        [ 0.5251753 ,  0.47482467, -0.4746756 ]]], dtype=float32)

In [35]:
preds.shape

(42131, 3)

In [37]:
preds_repeat = np.tile(preds, (1,23)).reshape(-1,23,3)
#np.repeat(preds, 23, axis=0).reshape(-1, 23, 3)
val_with_preds = np.concatenate((unadj_x_val, preds_repeat), axis=2)


In [39]:
col_names = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir', 'no_sack_prob', 'sack_prob', 'pred_time']

val_with_preds_df = pd.DataFrame(val_with_preds.reshape(-1, len(col_names)), columns=col_names)
val_with_preds_df.gameId = val_with_preds_df.gameId.astype('int64')


In [40]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time
0,2021101704,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,0.5461,0.4539,-0.730734
1,2021101704,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,0.5461,0.4539,-0.730734
2,2021101704,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,0.5461,0.4539,-0.730734
3,2021101704,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,0.5461,0.4539,-0.730734
4,2021101704,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,0.5461,0.4539,-0.730734


In [43]:
np.tile(y_val[:,1], (1,23)).reshape(-1).shape

(969013,)

In [45]:
val_with_preds_df['true_sack'] = np.tile(y_val[:,1], (1,23)).reshape(-1)
#np.repeat(y_val[:,1], 23)

In [46]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,2021101704,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,0.5461,0.4539,-0.730734,0.0
1,2021101704,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,0.5461,0.4539,-0.730734,0.0
2,2021101704,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,0.5461,0.4539,-0.730734,0.0
3,2021101704,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,0.5461,0.4539,-0.730734,0.0
4,2021101704,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,0.5461,0.4539,-0.730734,0.0


In [56]:
true_sack_df = val_with_preds_df.query("true_sack == 1")
true_sack_df.gameId = true_sack_df.gameId.astype('int64')

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_10841/1917819778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_sack_df.gameId = true_sack_df.gameId.astype('int64')


In [57]:
true_sack_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
186,2021101704,521.0,14.0,40171.0,1.0,87.20,32.46,1.13,0.05,165.88,145.97,0.455943,0.544057,0.279656,1.0
187,2021101704,521.0,14.0,41619.0,2.0,87.65,29.96,0.61,0.53,350.98,34.18,0.455943,0.544057,0.279656,1.0
188,2021101704,521.0,14.0,42444.0,1.0,87.43,25.36,2.73,3.13,150.93,159.21,0.455943,0.544057,0.279656,1.0
189,2021101704,521.0,14.0,42500.0,2.0,88.86,36.13,0.31,2.48,343.07,245.25,0.455943,0.544057,0.279656,1.0
190,2021101704,521.0,14.0,43045.0,2.0,88.04,28.79,1.22,0.83,355.52,57.22,0.455943,0.544057,0.279656,1.0


In [58]:
# test_game_id = 2021101704
# test_play_id = 746.0
# good example of probability increasing as defender comes near

test_game_id = 2021101704
test_play_id = 521.0

In [59]:

play_df = val_with_preds_df.query("gameId == @test_game_id and playId == @test_play_id")
print(f"play_df shape = {play_df.shape}")

play_df.head()

play_df shape = (805, 15)


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,2021101704,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,0.5461,0.4539,-0.730734,0.0
1,2021101704,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,0.5461,0.4539,-0.730734,0.0
2,2021101704,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,0.5461,0.4539,-0.730734,0.0
3,2021101704,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,0.5461,0.4539,-0.730734,0.0
4,2021101704,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,0.5461,0.4539,-0.730734,0.0


In [60]:
play_df.tail()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
800,2021101704,521.0,40.0,53434.0,2.0,67.80,17.65,9.17,2.54,340.70,339.06,0.892852,0.107148,-0.197637,0.0
801,2021101704,521.0,40.0,53542.0,1.0,71.16,26.76,7.45,1.02,275.54,350.18,0.892852,0.107148,-0.197637,0.0
802,2021101704,521.0,40.0,53619.0,2.0,89.11,30.11,1.71,1.22,334.03,243.99,0.892852,0.107148,-0.197637,0.0
803,2021101704,521.0,40.0,53953.0,1.0,73.80,14.45,7.63,0.85,59.42,332.12,0.892852,0.107148,-0.197637,0.0
804,2021101704,521.0,40.0,0.0,3.0,94.87,28.63,1.65,2.61,0.00,0.00,0.892852,0.107148,-0.197637,0.0


In [61]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [52]:
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

In [50]:
class AnimateFeature():
    
    # initialize variables we need. 
    # Example: Need to initialize the figure that the animation command will return
    def __init__(self, play_df, displayNumbers=False) -> None:
        
        self.MAX_FIELD_PLAYERS = 22
        self.start_x = 0
        self.stop_x = 120
        self.start_y = 53.3
        self.play_df = play_df
        self.frames_list = play_df.frameId.unique()
        self.games_df = pd.read_csv("~/Documents/Python/nfl-big-data-bowl-2023/data/games.csv")
        self.predictions = True
        self.displayNumbers = displayNumbers
        self.FIG_HEIGHT = 4
        self.FIG_WIDTH = 8
        
        fig, ax = plt.subplots(1, figsize=(self.FIG_WIDTH, self.FIG_HEIGHT))
        
        self.fig = fig
        self.field_ax = ax
        
        # create new axis for home, away, jersey
        self.ax_home = self.field_ax.twinx()
        self.ax_away = self.field_ax.twinx()
        self.ax_jersey = self.field_ax.twinx()
        self.ax_pred = self.field_ax.twinx()
        
        self.ani = animation.FuncAnimation(self.fig, self.update, frames=len(self.frames_list),
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
        
    # initialization function for animation call
    def setup_plot(self):

        endzones = True
        linenumbers = True
        
        # set axis limits
        self.set_axis_plots(self.field_ax, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_home, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_away, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_jersey, self.stop_x, self.start_y)
        self.set_axis_plots(self.ax_pred, self.stop_x, self.start_y)

        # set up colors and patches for field
        self.set_up_field(endzones, linenumbers)
        
        # create scatterplots on axis for data
        self.scat_field = self.field_ax.scatter([], [], s=50, color='orange')
        self.scat_home = self.ax_home.scatter([], [], s=50, color='blue')
        self.scat_away = self.ax_away.scatter([], [], s=50, color='red')
        
        # create box for prediction
        self.scat_pred = self.ax_pred.text(0, 0, '', fontsize = self.FIG_WIDTH, 
                                           bbox=dict(boxstyle="square", facecolor="white"),
                                           horizontalalignment = 'left', verticalalignment = 'top', c = 'black')
        
        # add direction stats and jersey numbers/names
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._a_dir_list = []
        self._a_or_list = []
        for _ in range(self.MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white'))
            self._scat_number_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            self._scat_name_list.append(self.ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            
            self._a_dir_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self.field_ax.add_patch(patches.Arrow(0, 0, 0, 0, color = 'k')))
        
        # return all axis plots that you want to update
        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def update(self, i):
        frame = self.frames_list[i]
        time_df = self.play_df.query("frameId == @frame")
        
        #time_df['team_indicator'] = self.add_team_indicator(time_df)
        #print(time_df)
        
        label_list = time_df.team_indicator.unique()
        #print(label_list)
        label1= label_list[0]
        label2 = label_list[1]
        label3 = label_list[2]
        
        # update each team/football x,y coordinates
        for label in label_list:
            label_data = time_df[time_df.team_indicator == label]
            
            if label == label1:
                self.scat_field.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label2:
                self.scat_home.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
            elif label == label3:
                self.scat_away.set_offsets(label_data[['adj_x','adj_y']].to_numpy())
                
        # add prediction
        if self.predictions != None:
            sack_prob = np.round(time_df.sack_prob.values[0], 3)
            time = np.round(time_df.pred_time.values[0], 3)
            true_sack = time_df.true_sack.values[0]
            set_str = f"Sack = {true_sack}, P(sack) = {sack_prob}, time = {time}"
            self.scat_pred.set_text(set_str)
        
        #add direction and jersey info
        jersey_df = time_df[time_df.nflId != 0].reset_index()
        
        for (index, row) in jersey_df.iterrows():
            #self._scat_jersey_list[index].set_position((row.x, row.y))
            #self._scat_jersey_list[index].set_text(row.position)
            
            if self.displayNumbers:
                self._scat_number_list[index].set_position((row.adj_x, row.adj_y))
                self._scat_number_list[index].set_text(int(row.nflId))
            #self._scat_name_list[index].set_position((row.x, row.y-1.9))
            #self._scat_name_list[index].set_text(row.displayName.split()[-1])
            
            player_orientation_rad = self.deg_to_rad(self.convert_orientation(row.adj_o))
            player_direction_rad = self.deg_to_rad(self.convert_orientation(row.adj_dir))
            player_speed = row.s
            
            player_vel = np.array([np.real(self.polar_to_z(player_speed, player_direction_rad)), np.imag(self.polar_to_z(player_speed, player_direction_rad))])
            player_orient = np.array([np.real(self.polar_to_z(2, player_orientation_rad)), np.imag(self.polar_to_z(2, player_orientation_rad))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_vel[0], player_vel[1], color = 'k'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self.field_ax.add_patch(patches.Arrow(row.adj_x, row.adj_y, player_orient[0], player_orient[1], color = 'grey', width = 2))
                

        return (self.scat_field, self.scat_home, self.scat_away, self.ax_pred, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
    
    def set_up_field(self, endzones=True, linenumbers=True) -> None:
        yard_numbers_size = self.fig.get_size_inches()[0]*1.5

        # color field 
        rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                                    edgecolor='r', facecolor='darkgreen', zorder=0)
        self.field_ax.add_patch(rect)

        # plot
        self.field_ax.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
                    80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
                    [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
                    53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
                    color='white')

        # Endzones
        if endzones:
            ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                    linewidth=0.1,
                                    edgecolor='r',
                                    facecolor='blue',
                                    alpha=0.2,
                                    zorder=0)
            self.field_ax.add_patch(ez1)
            self.field_ax.add_patch(ez2)
            
        if endzones:
            hash_range = range(11, 110)
        else:
            hash_range = range(1, 120)

        # add hashes
        for x in hash_range:
            self.field_ax.plot([x, x], [0.4, 0.7], color='white')
            self.field_ax.plot([x, x], [53.0, 52.5], color='white')
            self.field_ax.plot([x, x], [22.91, 23.57], color='white')
            self.field_ax.plot([x, x], [29.73, 30.39], color='white')
            
        # add linenumbers
        if linenumbers:
                for x in range(20, 110, 10):
                    numb = x
                    if x > 50:
                        numb = 120 - x
                    self.field_ax.text(x, 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white')
                    self.field_ax.text(x - 0.95, 53.3 - 5, str(numb - 10),
                            horizontalalignment='center',
                            fontsize=yard_numbers_size,  # fontname='Arial',
                            color='white', rotation=180)

        self.field_ax.set_xlim(self.start_x, self.stop_x)
        self.field_ax.set_ylim(0, self.start_y)
        self.field_ax.set_xticks(range(self.start_x,self.stop_x, 10))

    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        

In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
